In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data

import time, json, datetime 
from tqdm import tqdm

import numpy as np 
import pandas as pd 
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
class DeepFM(nn.Module):
    def __init__(self, cate_fea_nuniqs, nume_fea_size=0, emb_size=8, 
                 hid_dims=[256, 128], num_classes=1, dropout=[0.2, 0.2]): 
        """
        cate_fea_nuniqs: 类别特征的唯一值个数列表，也就是每个类别特征的vocab_size所组成的列表
        nume_fea_size: 数值特征的个数，该模型会考虑到输入全为类别型，即没有数值特征的情况 
        emb_size：嵌入层的纬度，也就是隐向量的k的值，emb层把原始的稀疏向量变成一个稠密的隐向量
        hid_dims：隐藏层（也即全连接层的纬度？）256,128代表全连接层有两层，第一层有256个神经元，第二层12个
        num_classes：默认等于1的话就是表示一个二分类问题，最后返回的结果就是一个标量，表示结果为1的概率
        dropout：防止过拟合的方法，比如说有256个神经元，0.2表示会随机的丢掉百分之20的神经元，具体是用一个256维的向量有0.8的都是1，其他的是0，与输入的向量做内积，将得到的向量再去链接
        0.2,0.2表示输入层会dropout0.2，每个隐藏层也是0.2
        
        ？？self具体是什么呢？
        """
        super().__init__()
        self.cate_fea_size = len(cate_fea_nuniqs) # cate_fea_nuniqs是一个列表，里面是输入数据中类别型的特征列
        self.nume_fea_size = nume_fea_size
        
        """FM部分"""
        # 一阶
        if self.nume_fea_size != 0: # 如果有数值型的特征
            self.fm_1st_order_dense = nn.Linear(self.nume_fea_size, 1)  # 数值特征的一阶表示
            """
            linear层具体是怎么实现的，干什么？
            这个地方创建了一个linear层对象，
            self.fm_1st_order_dense是self类的一个属性
            """
        self.fm_1st_order_sparse_emb = nn.ModuleList([
            nn.Embedding(voc_size, 1) for voc_size in cate_fea_nuniqs])  # 类别特征的一阶表示
        """
        self.fm_1st_order_sparse_emb是一个列表，其中的每个元素都是一个nn.Embedding对象
        voc_size表示每个类别型特征的可能取值数量
        nn.Embedding(voc_size, 1)函数创建一个nn.Embedding对象
        """
        
        # 二阶
        self.fm_2nd_order_sparse_emb = nn.ModuleList([
            nn.Embedding(voc_size, emb_size) for voc_size in cate_fea_nuniqs])  # 类别特征的二阶表示
        """
        同样是一个列表，每一项都是一个embedding对象，但是这个对象是一个n*k的矩阵
        """
        
        """DNN部分"""
        self.all_dims = [self.cate_fea_size * emb_size] + hid_dims # 合成一个新向量，第一个元素是品类特征数*
        self.dense_linear = nn.Linear(self.nume_fea_size, self.cate_fea_size * emb_size)  # 数值特征的维度变换到FM输出维度一致
        self.relu = nn.ReLU()
        # for DNN 
        for i in range(1, len(self.all_dims)):
            setattr(self, 'linear_'+str(i), nn.Linear(self.all_dims[i-1], self.all_dims[i]))
            setattr(self, 'batchNorm_' + str(i), nn.BatchNorm1d(self.all_dims[i]))
            setattr(self, 'activation_' + str(i), nn.ReLU())
            setattr(self, 'dropout_'+str(i), nn.Dropout(dropout[i-1]))
        # for output 
        self.dnn_linear = nn.Linear(hid_dims[-1], num_classes)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, X_sparse, X_dense=None):
        """
        X_sparse: 类别型特征输入  [bs, cate_fea_size]
        X_dense: 数值型特征输入（可能没有）  [bs, dense_fea_size]
        """
        
        """FM 一阶部分"""
        fm_1st_sparse_res = [emb(X_sparse[:, i].unsqueeze(1)).view(-1, 1) 
                             for i, emb in enumerate(self.fm_1st_order_sparse_emb)]
        fm_1st_sparse_res = torch.cat(fm_1st_sparse_res, dim=1)  # [bs, cate_fea_size]
        fm_1st_sparse_res = torch.sum(fm_1st_sparse_res, 1,  keepdim=True)  # [bs, 1]
        
        if X_dense is not None:
            fm_1st_dense_res = self.fm_1st_order_dense(X_dense) 
            fm_1st_part = fm_1st_sparse_res + fm_1st_dense_res
        else:
            fm_1st_part = fm_1st_sparse_res   # [bs, 1]
        
        """FM 二阶部分"""
        fm_2nd_order_res = [emb(X_sparse[:, i].unsqueeze(1)) for i, emb in enumerate(self.fm_2nd_order_sparse_emb)]
        fm_2nd_concat_1d = torch.cat(fm_2nd_order_res, dim=1)  # [bs, n, emb_size]  n为类别型特征个数(cate_fea_size)
        
        # 先求和再平方
        sum_embed = torch.sum(fm_2nd_concat_1d, 1)  # [bs, emb_size]
        square_sum_embed = sum_embed * sum_embed    # [bs, emb_size]
        # 先平方再求和
        square_embed = fm_2nd_concat_1d * fm_2nd_concat_1d  # [bs, n, emb_size]
        sum_square_embed = torch.sum(square_embed, 1)  # [bs, emb_size]
        # 相减除以2 
        sub = square_sum_embed - sum_square_embed  
        sub = sub * 0.5   # [bs, emb_size]
        
        fm_2nd_part = torch.sum(sub, 1, keepdim=True)   # [bs, 1]
        
        """DNN部分"""
        dnn_out = torch.flatten(fm_2nd_concat_1d, 1)   # [bs, n * emb_size]
        
        if X_dense is not None:
            dense_out = self.relu(self.dense_linear(X_dense))   # [bs, n * emb_size]
            dnn_out = dnn_out + dense_out   # [bs, n * emb_size]
        
        for i in range(1, len(self.all_dims)):
            dnn_out = getattr(self, 'linear_' + str(i))(dnn_out)
            dnn_out = getattr(self, 'batchNorm_' + str(i))(dnn_out)
            dnn_out = getattr(self, 'activation_' + str(i))(dnn_out)
            dnn_out = getattr(self, 'dropout_' + str(i))(dnn_out)
        
        dnn_out = self.dnn_linear(dnn_out)   # [bs, 1]
        out = fm_1st_part + fm_2nd_part + dnn_out   # [bs, 1]
        out = self.sigmoid(out)
        return out

## 数据预处理

In [ ]:
data = pd.read_csv('data/criteo_sample_50w.csv')

dense_features = [f for f in data.columns.tolist() if f[0] == "I"]
sparse_features = [f for f in data.columns.tolist() if f[0] == "C"]
"""
分别是两个向量，
一个是数据型的向量，内容是数据类型向量的标题
一个是类别型的向量，内容是类别类型向量的标题
"""
data[sparse_features] = data[sparse_features].fillna('-10086', )
data[dense_features] = data[dense_features].fillna(0, )
target = ['label']
"""
对数据集中的缺失值进行填充
"""
## 类别特征labelencoder
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
## 数值特征标准化
for feat in tqdm(dense_features):
    mean = data[feat].mean()
    std = data[feat].std() # 计算特征的标准差
    data[feat] = (data[feat] - mean) / (std + 1e-12)   # 防止除零
    
"""
把对数据进一步处理，
首先是类别的数据：
先实例化一个LabelEncoder对象，然后调用方法fit_transform方法，传入一个特征列向量，该方法会把类别用数字代替，然后返回列向量，更新
上下的区别在于下面加了一个tqdm()方法,就是让这个过程一进度条的方式可视化出来，可以直接删掉，在大型数据处理中不会用，这个过程会造成损耗
下面是数据类型的：目的是使得每个特征的均值为0，方差为1。
先求每一个的均值，方差，然后计算处理
"""
pass
# print(data.shape)
# print(data.head())

100%|██████████| 13/13 [00:00<00:00, 94.81it/s]


In [24]:
train, valid = train_test_split(data, test_size=0.2, random_state=2020)
"""
random_state是一个随机数的种子，用来随机分割数据集，当他为一个确定的数值的时候，就能够保证每次跑程序，数据集被分割的一样，是可复现的，
"""
# print(train.shape, valid.shape)

train_dataset = Data.TensorDataset(torch.LongTensor(train[sparse_features].values), 
                                   torch.FloatTensor(train[dense_features].values),
                                   torch.FloatTensor(train['label'].values),)

"""
训练集转化，讲训练集中的类别向量转换为LongTensor,数值向量转换为FloatTensor,label是点击次数，也作为FloatTensor,然后合并
"""

train_loader = Data.DataLoader(dataset=train_dataset, batch_size=2048, shuffle=True)
"""
得到了训练集进一步处理，将所有的训练数据变成一个一个的batch，shuffle是指每一个epoch都会重新随机分配数据形成新的batch组

下面对测试集的数据也是同样的处理方法
"""
valid_dataset = Data.TensorDataset(torch.LongTensor(valid[sparse_features].values), 
                                   torch.FloatTensor(valid[dense_features].values),
                                   torch.FloatTensor(valid['label'].values),)
valid_loader = Data.DataLoader(dataset=valid_dataset, batch_size=4096, shuffle=False)



device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')



# print(device)
cate_fea_nuniqs = [data[f].nunique() for f in sparse_features]

"""
首先sparse_features是类别向量的列标题，因此是循环每一个类别向量，
cate_fea_nuniqs 是一个向量，每一个元素是每一个类别向量的不同的元素的个数，表示这一列有多少种不同的类别
"""

model = DeepFM(cate_fea_nuniqs, nume_fea_size=len(dense_features))
"""
声明一个deepFM对象，需要传入刚刚得到的，一个每个类别特征所包含的不同的种类数的向量，一个数值类型的特征数
"""


model.to(device)# 运行模型


loss_fcn = nn.BCELoss()  # Loss函数
loss_fcn = loss_fcn.to(device) # 将损失函数转移到指定的设备上

optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)
"""
第一个是模型的参数
lr学习率，weight_decay是l2正则化的系数
整个optimizer是基于梯度下降算法的优化器
"""

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)
"""
定义一个学习率调度器，optimizer是优化器，step_size是更新学习率的间隔，=1表示每一个epoch就更新一次，gamma是倍率，即每次更新，学习率都乘上0.8
"""



# 打印模型参数
def get_parameter_number(model):
    total_num = sum(p.numel() for p in model.parameters())
    trainable_num = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return {'Total': total_num, 'Trainable': trainable_num}
print(get_parameter_number(model))

# 定义日志（data文件夹下，同级目录新建一个data文件夹）
def write_log(w):
    file_name = 'data/' + datetime.date.today().strftime('%m%d')+"_{}.log".format("deepfm")
    t0 = datetime.datetime.now().strftime('%H:%M:%S')
    info = "{} : {}".format(t0, w)
    print(info)
    with open(file_name, 'a') as f: 
        f.write(info + '\n') 

{'Total': 7037251, 'Trainable': 7037251}


In [6]:
def train_and_eval(model, train_loader, valid_loader, epochs, device):
    best_auc = 0.0
    for _ in range(epochs):
        """训练部分"""
        model.train() #开启训练模式，在这个模式下，模型会使用dropout，batch，normalization等一些有利于模型的方法，model.eval()就是不会
        print("Current lr : {}".format(optimizer.state_dict()['param_groups'][0]['lr']))
        write_log('Epoch: {}'.format(_ + 1))
        train_loss_sum = 0.0
        start_time = time.time()
        
        
        for idx, x in enumerate(train_loader):
            cate_fea, nume_fea, label = x[0], x[1], x[2]
            cate_fea, nume_fea, label = cate_fea.to(device), nume_fea.to(device), label.float().to(device)
            pred = model(cate_fea, nume_fea).view(-1)
            loss = loss_fcn(pred, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss_sum += loss.cpu().item()
            if (idx+1) % 50 == 0 or (idx + 1) == len(train_loader):
                write_log("Epoch {:04d} | Step {:04d} / {} | Loss {:.4f} | Time {:.4f}".format(
                          _+1, idx+1, len(train_loader), train_loss_sum/(idx+1), time.time() - start_time))
        scheduler.step()
        
        
        """推断部分"""
        model.eval()
        with torch.no_grad():
            valid_labels, valid_preds = [], []
            for idx, x in tqdm(enumerate(valid_loader)):
                cate_fea, nume_fea, label = x[0], x[1], x[2]
                cate_fea, nume_fea = cate_fea.to(device), nume_fea.to(device)
                pred = model(cate_fea, nume_fea).reshape(-1).data.cpu().numpy().tolist()
                valid_preds.extend(pred)
                valid_labels.extend(label.cpu().numpy().tolist())
        cur_auc = roc_auc_score(valid_labels, valid_preds)
        if cur_auc > best_auc:
            best_auc = cur_auc
            torch.save(model.state_dict(), "data/deepfm_best.pth")
        write_log('Current AUC: %.6f, Best AUC: %.6f\n' % (cur_auc, best_auc))
        

train_and_eval(model, train_loader, valid_loader, 30, device)

Current lr : 0.005
22:00:22 : Epoch: 1
22:00:26 : Epoch 0001 | Step 0050 / 196 | Loss 18.9375 | Time 3.7620
22:00:29 : Epoch 0001 | Step 0100 / 196 | Loss 16.4242 | Time 7.0448
22:00:32 : Epoch 0001 | Step 0150 / 196 | Loss 14.1202 | Time 10.5061


2it [00:00, 18.02it/s]

22:00:35 : Epoch 0001 | Step 0196 / 196 | Loss 12.0842 | Time 13.5648


25it [00:01, 13.57it/s]


22:00:37 : Current AUC: 0.666958, Best AUC: 0.666958

Current lr : 0.004
22:00:37 : Epoch: 2
22:00:41 : Epoch 0002 | Step 0050 / 196 | Loss 2.8049 | Time 3.3196
22:00:44 : Epoch 0002 | Step 0100 / 196 | Loss 2.2271 | Time 6.5872
22:00:47 : Epoch 0002 | Step 0150 / 196 | Loss 1.8805 | Time 9.9019


...

22:07:36 : Current AUC: 0.733417, Best AUC: 0.778540

Current lr : 7.73712524553364e-06
22:07:36 : Epoch: 30
22:07:39 : Epoch 0030 | Step 0050 / 196 | Loss 0.2684 | Time 3.2587
22:07:42 : Epoch 0030 | Step 0100 / 196 | Loss 0.2675 | Time 6.5317
22:07:45 : Epoch 0030 | Step 0150 / 196 | Loss 0.2683 | Time 9.7743


2it [00:00, 17.58it/s]

22:07:49 : Epoch 0030 | Step 0196 / 196 | Loss 0.2685 | Time 12.8603


25it [00:01, 13.96it/s]

22:07:50 : Current AUC: 0.733560, Best AUC: 0.778540

